# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [202]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [203]:
csv_path = "../WeatherPy/CitiesWeather.csv"
cityweather = pd.read_csv(csv_path, encoding="utf -8")
cityweather

,City,Country,Latitude,Longitude,Date,Temperature,Humidity,Cloudiness,Wind Speed
0,kang,BW,-23.6752,22.7876,2021-01-14 16:29:01,30.77,33.0,69.0,2.41
1,casablanca,MA,33.5928,-7.6192,2021-01-14 16:27:33,14.51,55.0,40.0,2.06
2,huarmey,PE,-10.0681,-78.1522,2021-01-14 16:29:00,23.74,72.0,100.0,3.89
3,khatanga,RU,71.9667,102.5000,2021-01-14 16:29:03,-44.88,77.0,91.0,0.94
4,busselton,AU,-33.6500,115.3333,2021-01-14 16:29:04,15.15,73.0,35.0,0.45
...,...,...,...,...,...,...,...,...,...
577,chunhuhub,MX,19.5500,-88.6833,2021-01-14 16:35:40,22.60,83.0,98.0,1.29
578,jackson,US,42.4165,-122.8345,2021-01-14 16:35:41,3.77,93.0,90.0,0.46
579,tunceli,TR,39.1083,39.5471,2021-01-14 16:35:41,5.27,81.0,75.0,3.60
580,kangaatsiaq,GL,68.3065,-53.4641,2021-01-14 16:35:42,-8.99,96.0,100.0,1.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [205]:
print(cityweather["Latitude"].dtype)
print(cityweather["Longitude"].dtype)
print(cityweather["Humidity"].dtype)

float64
float64
float64


In [206]:
# Store 'Lat' and 'Lng' into  locations 
locations = cityweather[["Latitude", "Longitude"]]

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#census_data_complete = census_data_complete.dropna()
humidity = cityweather["Humidity"]

print(len(locations))
print(len(humidity))

582
582


In [207]:
#locations = earthquake_df[['latitude', 'longitude']]
#weights = earthquake_df['magnitude']
#fig = gmaps.figure()
#fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
#fig

In [208]:
#import gmaps
#gmaps.configure(api_key=g_key)
#fig = gmaps.figure()
#locations = [(46.1, 5.2), (46.2, 5.3), (46.3, 5.4)]
#fig.add_layer(gmaps.heatmap_layer(locations))
#fig

In [209]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#fig = gmaps.figure(center=(46.0, -5.0), zoom_level=8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [210]:
cityweather.columns

Index(['City', 'Country', 'Latitude', 'Longitude', 'Date', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind Speed'],
      dtype='object')

In [211]:
cityweather["Wind Speed"].max()

17.23

In [212]:
#A max temperature lower than 30 degrees but higher than 20 = celsius.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is id

holiday = cityweather.loc[(cityweather["Temperature"]>20) & (cityweather["Temperature"]<30), :]

#cond16 = (df['daychange']<8) & (df['daychange']>2
holiday = holiday.loc[cityweather["Wind Speed"] < 10, :]
holiday = holiday.loc[cityweather["Cloudiness"] == 0, :]
holiday.index=range(0,len(holiday))

holiday

,City,Country,Latitude,Longitude,Date,Temperature,Humidity,Cloudiness,Wind Speed
0,cape town,ZA,-33.9258,18.4232,2021-01-14 16:26:39,27.69,60.0,0.0,6.17
1,pimentel,PE,-6.8367,-79.9342,2021-01-14 16:29:47,28.00,51.0,0.0,5.66
2,pisco,PE,-13.7000,-76.2167,2021-01-14 16:30:04,21.86,78.0,0.0,4.63
3,carnarvon,AU,-24.8667,113.6333,2021-01-14 16:30:24,26.00,89.0,0.0,9.26
4,dolores,AR,-36.3132,-57.6792,2021-01-14 16:30:34,28.16,23.0,0.0,7.59
5,hermanus,ZA,-34.4187,19.2345,2021-01-14 16:30:37,28.33,66.0,0.0,2.24
6,chuy,UY,-33.6971,-53.4616,2021-01-14 16:31:05,24.44,45.0,0.0,6.42
7,mulege,MX,26.8833,-111.9833,2021-01-14 16:31:20,20.97,21.0,0.0,2.08
8,dalby,AU,-27.1833,151.2667,2021-01-14 16:31:36,20.56,78.0,0.0,1.95
9,micheweni,TZ,-4.9667,39.8333,2021-01-14 16:31:52,26.66,78.0,0.0,6.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [213]:
column_names = ["Hotel Name", "Lat", "Lng", "City", "Country"]

hotel_df = pd.DataFrame(columns = column_names)
hotel_df

,Hotel Name,Lat,Lng,City,Country


In [214]:
#Test adding values to dataframe
hotel_df.loc[0] = 10.    #['name' + str(i)] + list(randint(10, size=2))
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,10.0,10.0,10.0,10.0,10.0


* Set parameters to search for hotels with 5000 meters.

In [215]:
#TEST RESPONSE and json check
# geocoordinates

target_coordinates = "43.6187102, -116.2146068"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.6139805,
                    "lng": -116.2037792
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.61524747989272,
                        "lng": -116.2022788701073
                    },
                    "southwest": {
                        "lat": 43.61254782010728,
                        "lng": -116.2049785298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "The Grove Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 553,
                    "html_attributions": [
                        "<a 

In [216]:
# geocoordinates
#target_coordinates = "43.6187102, -116.2146068"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params)


* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.

In [217]:
#WORKING ON THIS ONE
# Loop through the list of coordinates and perform a request for data on each
for c in range(0, len(holiday)):
#for c in range(0, 10):
    
    lat = holiday["Latitude"][c]
    long = holiday["Longitude"][c]
    
    #print(lat, long)
    
    params = {
    "location": f"{lat}, {long}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
    
    #print(params)
    
    response = requests.get(base_url, params=params)
    #print(response)
    
    # convert response to json
    try:
        #print("Processing Record " + str((holiday.index[c])+1) + " | " +  holiday["City"])
        places_data = response.json()
                
        hotelname = places_data["results"][0]["name"]
        latnew = places_data["results"][0]["geometry"]["location"]["lat"]
        longnew = places_data["results"][0]["geometry"]["location"]["lng"]
                
        print("Processing " + hotelname)
        #print(latnew)
        #print(latnew)
        #print(holiday["City"][c])
        #print(holiday["Country"][c])
        
        
        hotel_df.loc[c] = hotelname
        hotel_df["Lat"][c] = latnew
        hotel_df["Lng"][c] = longnew
        hotel_df["City"][c] = holiday["City"][c]
        hotel_df["Country"][c] = holiday["Country"][c]
        
                
        
        
        #hotel_df["Hotel Name"]['c'] = places_data["results"][0]["name"]
   
    except IndexError:
        print("No information - Skipping")
    
    
   # Print the name and address of the first restaurant that appears
    #print(places_data["results"][0]["name"])
    #print(places_data["results"][0]["vicinity"])
    


Processing Atlanticview Cape Town Boutique Hotel
Processing Hotel La Posada Del Ingles
Processing HOSTAL MIRAMAR
Processing Carnarvon Sea Change Apartments
Processing HOTEL TERMAL HOWARD JOHNSON DOLORES
Processing Misty Waves Boutique Hotel
Processing Turis Firper Hotel
Processing Hotel Serenidad
Processing Drovers Motor Inn
No information - Skipping
Processing Hotel Cabildo
Processing Les Embruns Du Baril
Processing Itaimbé Palace Hotel
Processing BOB FISHING CLUB BIJAGOS
Processing DoubleTree by Hilton Perth Northbridge
Processing SAWTELL BEACH SHACKS 1 & 2
Processing Osmani Hotel
Processing HYATT SALALAH HOTEL
Processing Estepa Hotel
Processing Seashells Mandurah
Processing Lucania Palazzo Hotel
Processing LHACIENDA976
Processing Al Camar Lodge
Processing Gran Hotel Mercedes


In [218]:
hotel_df.index=range(0,len(hotel_df))
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Atlanticview Cape Town Boutique Hotel,-33.9596,18.3853,cape town,ZA
1,Hotel La Posada Del Ingles,-6.79784,-79.8868,pimentel,PE
2,HOSTAL MIRAMAR,-13.7274,-76.2223,pisco,PE
3,Carnarvon Sea Change Apartments,-24.8841,113.655,carnarvon,AU
4,HOTEL TERMAL HOWARD JOHNSON DOLORES,-36.3137,-57.6527,dolores,AR
5,Misty Waves Boutique Hotel,-34.4214,19.2366,hermanus,ZA
6,Turis Firper Hotel,-33.6892,-53.449,chuy,UY
7,Hotel Serenidad,26.898,-111.959,mulege,MX
8,Drovers Motor Inn,-27.1871,151.267,dalby,AU
9,Hotel Cabildo,-34.6449,-58.6198,moron,AR


* Plot markers on top of the heatmap

In [219]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [220]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure

fig.add_layer(markers)
fig


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))